In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets

In [33]:
# Define a transform to convert the images to tensors and normalize them
transform = transforms.Compose([
    transforms.Resize((30, 30)), # Resize the image to 30x30 pixels
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the datasets with ImageFolder
train_dataset = datasets.ImageFolder(root='archive/Training', transform=transform)
test_dataset = datasets.ImageFolder(root='archive/Testing', transform=transform)

# Create the dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=8)

In [15]:
# the size of the dataset and the shape of images
num_images = len(train_dataset)
image_shape = next(iter(train_loader))[0].shape[1:]  # CxHxW

image_shape

torch.Size([3, 30, 30])

In [16]:
# Initialize numpy arrays for storing data and labels
data_all = np.zeros((num_images,) + image_shape, dtype=np.float32)
labels_all = np.zeros(num_images, dtype=np.int64)

# Iterate over the DataLoader and fill the arrays
start_idx = 0
for images, labels in train_loader:
    end_idx = start_idx + images.size(0)
    data_all[start_idx:end_idx] = images.numpy()
    labels_all[start_idx:end_idx] = labels.numpy()
    start_idx = end_idx
    
# Now data_all is a NumPy array containing all images
# and labels_all contains all corresponding labels
# {'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}

In [23]:
#splitting the training dataset into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(data_all, labels_all, test_size=0.2, random_state=42)

#printing the shape of the training and testing datasets
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(4569, 3, 30, 30) (1143, 3, 30, 30) (4569,) (1143,)


In [30]:
class BrainTumorClassifier(nn.Module):
    def __init__(self):
        super(BrainTumorClassifier, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        
        # Pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 3 * 3, 512)  # The input size here should match the output size of the last conv layer
        self.fc2 = nn.Linear(512, 4)  # Output layer with 4 classes
        
        # Activation and dropout
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # Convolutional layers with ReLU and pooling
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        
        # Flattening the output for the dense layer
        x = x.view(x.size(0), -1)
        
        # Fully connected layer with ReLU and dropout
        x = self.dropout(self.relu(self.fc1(x)))
        
        # Output layer with softmax activation
        x = self.softmax(self.fc2(x))
        
        return x
    
def to_one_hot(labels, num_classes):
    return torch.eye(num_classes)[labels]

In [31]:
y_train_one_hot = to_one_hot(y_train, num_classes=4)
y_test_one_hot = to_one_hot(y_test, num_classes=4)

y_train_one_hot

tensor([[1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        ...,
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.]])

In [47]:
# Instantiate the model
model = BrainTumorClassifier()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()  # CrossEntropyLoss applies softmax internally
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

device = torch.device("mps")
model = model.to(device)

# Define the number of epochs for training
num_epochs = 25

# Start the training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # Training
    for data in train_loader:
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_accuracy = 100 * correct_train / total_train
    train_loss = running_loss / len(train_loader)
    
    # Validation
    model.eval()  # Set the model to evaluation mode
    running_loss_val = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss_val += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
    
    val_accuracy = 100 * correct_val / total_val
    val_loss = running_loss_val / len(test_loader)

    # Print statistics
    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

print('Finished Training')

Epoch 1/25, Train Loss: 1.1527, Train Accuracy: 57.77%, Val Loss: 1.0992, Val Accuracy: 62.70%
Epoch 2/25, Train Loss: 1.0383, Train Accuracy: 70.13%, Val Loss: 1.0847, Val Accuracy: 64.07%
Epoch 3/25, Train Loss: 0.9821, Train Accuracy: 75.70%, Val Loss: 1.0092, Val Accuracy: 72.16%
Epoch 4/25, Train Loss: 0.9470, Train Accuracy: 79.80%, Val Loss: 1.0029, Val Accuracy: 73.15%
Epoch 5/25, Train Loss: 0.9187, Train Accuracy: 82.42%, Val Loss: 0.9664, Val Accuracy: 77.27%
Epoch 6/25, Train Loss: 0.9017, Train Accuracy: 84.21%, Val Loss: 0.9737, Val Accuracy: 76.20%
Epoch 7/25, Train Loss: 0.8956, Train Accuracy: 84.72%, Val Loss: 0.9344, Val Accuracy: 80.32%
Epoch 8/25, Train Loss: 0.8801, Train Accuracy: 86.27%, Val Loss: 0.9304, Val Accuracy: 80.40%
Epoch 9/25, Train Loss: 0.8624, Train Accuracy: 88.15%, Val Loss: 0.9106, Val Accuracy: 82.84%
Epoch 10/25, Train Loss: 0.8431, Train Accuracy: 90.21%, Val Loss: 0.9012, Val Accuracy: 84.06%
Epoch 11/25, Train Loss: 0.8340, Train Accuracy: 